### Government Sector 

In [1]:
%config Completer.use_jedi = False
import sys
import json
from bs4 import BeautifulSoup
sys.path.append('../src')

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

### Number of social security beneficiaries

Time series downloaded from here: https://www.ssa.gov/OACT/ProgData/icp.html

Long-term data were downloaded manually and stored locally. 

In [2]:
# Manually retrieved and locally stored data
raw = pd.read_csv('/home/brian/Documents/uschartbook/data/ssa_beneficiaries.csv', 
                  index_col='Date', parse_dates=True)

# Recent monthly data
url = 'https://www.ssa.gov/cgi-bin/currentpay.cgi'
r = pd.read_html(url)
df = r[2]
df.columns = [i[1] for i in df.columns] 
df = df.rename({'End of': 'Date'}, axis=1)
df.index = pd.to_datetime(df['Date'])
df = df.drop('Date', axis=1)

# Combine long-term and monthly data
res = pd.concat([raw, df]).drop_duplicates(keep='last')
res.to_csv(data_dir / 'ss_beneficiaries.csv', index_label='Date')

### Medicare enrollment

Long term data found on census.gov stored locally. Data from 2013 onward retrieved from changing link.

In [3]:
# Manually retrieved and locally stored data
raw = pd.read_csv('/home/brian/Documents/uschartbook/data/medicare.csv', 
                  index_col='Date', parse_dates=True)
raw.index = raw.index + pd.DateOffset(months=6)

# Recent data 
url = 'https://catalog.data.gov/dataset/medicare-monthly-enrollment'
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')
links = [link.get('href') for link in soup('a')]
furl = [i for i in links if i.endswith('csv')][0]
df = pd.read_csv(furl)
res = (df.query('BENE_STATE_DESC == "National" and MONTH != "Year"'))
res.index = [pd.to_datetime(f'{i[1].MONTH} {i[1].YEAR}') 
             for i in res.iterrows()]
data = res['TOT_BENES']
data = res['TOT_BENES']
data.name = 'Total'
data.index.name = 'Date'

# Combine long-term and monthly data
comb = pd.concat([raw['Total'], data], axis=0).astype('float')
comb.to_csv(data_dir / 'medicare_enrollment.csv', index_label='Date')

### Medicaid and CHIP enrollment data

Long-term data stored locally

Recent data from csv file here: https://data.medicaid.gov/dataset/6165f45b-ca93-5bb5-9d06-db29c692a360

In [4]:
# Manually retrieved and locally stored data
raw = pd.read_csv('/home/brian/Documents/uschartbook/data/medicaid.csv', 
                  index_col='Date', parse_dates=True)
raw.index = raw.index + pd.DateOffset(months=6)

# Retrieve recent data and combine with long-term data
url = ('https://data.medicaid.gov/sites/default/files/uploaded_resources/'+
       'StateMedicaidandCHIPApplicationsEligibilityDeterminationsandEnrollmentData'+
       'August2023.csv')

df = pd.read_csv(url)
dates = df.report_date.unique()
upd = df.query('preliminary_updated == "U"')
prl = df.query('report_date == @dates[-1]')
data = pd.concat([upd, prl]).dropna(subset='total_medicaid_enrollment')
data['date'] = pd.to_datetime(data['report_date'])
tot = pd.concat([raw['Total'], 
                 (data.groupby('date')['total_medicaid_and_chip_enrollment'].sum()
                  / 1_000_000).loc['2022':]])
tot.to_csv(data_dir / 'medicaid_enrollment.csv', index_label='Date')

In [5]:
df = pd.read_csv(url)
dates = df.report_date.unique()
upd = df.query('preliminary_updated == "U"')
prl = df.query('report_date == @dates[-1]')
data = pd.concat([upd, prl]).dropna(subset='total_medicaid_enrollment')
data['date'] = pd.to_datetime(data['report_date'])
tot = pd.concat([raw['Total'], 
                 (data.groupby('date')['total_medicaid_and_chip_enrollment'].sum()
                  / 1_000_000).loc['2022':]])
tot.to_csv(data_dir / 'medicaid_enrollment.csv', index_label='Date')

### Combine SS, Medicare, Medicaid and calculation share of population

In [6]:
# Social Security Retrieve data
res = pd.read_csv(data_dir / 'ss_beneficiaries.csv', index_col='Date', 
                  parse_dates=True)

# Population
cols = ['B230RC']
dfm = (pd.read_csv(data_dir/'nipa20600.csv', index_col='date', 
                  parse_dates=True)[cols])
pop = dfm['B230RC']
# Some months not available in nipa tables yet
pope = pd.read_csv(data_dir / 'pop_est_bea.csv', 
                   index_col='date', parse_dates=True)
pop = (pd.concat([pop, (pope['POP'] / 1000)]).reset_index()
         .drop_duplicates(subset='date', keep='first')
         .set_index('date')[0])
sh = (res['Number'] / pop / 10).dropna()

ltdt = dtxt(res.index[-1])['mon1']
v10 = res.loc['2010', 'Number'].mean() / 1_000_000
sh10 = value_text(sh.loc['2010'].mean(), 'plain')
vlt = res.Number.iloc[-1] / 1_000_000
shlt = value_text(sh.iloc[-1], 'plain')
txt1 = ('The number of Social Security beneficiaries has increased '+
        f'from {v10:.1f} million in 2010, or {sh10} '+
        f'of the population, to {vlt:.1f} million in {ltdt}, '+
        f'equivalent to {shlt} of the population. ')


# Medicare Retrieve data
comb = pd.read_csv(data_dir / 'medicare_enrollment.csv', index_col='Date', 
                  parse_dates=True)
sh2 = comb.divide(pop, axis=0).dropna() / 10


# Retrieve Medicaid data
tot = pd.read_csv(data_dir / 'medicaid_enrollment.csv', index_col='Date', 
                  parse_dates=True)['0']
sh3 = tot.divide(pop, axis=0).dropna() * 100_000


# Combine data
final = pd.concat([sh, sh2['Total'], sh3], axis=1)
final.columns = ['SS', 'Medicare', 'Medicaid']
final.loc['1989':].to_csv(data_dir / 'prog_sh.csv', index_label='date')

are10 = value_text(final.loc['2010', 'Medicare'].mean(), 'plain')
arelt = value_text(final.Medicare.dropna().iloc[-1], 'plain')
arevlt = comb.Total.iloc[-1] / 1_000_000
areltdt = dtxt(comb.index[-1])['mon1']
aid10 = value_text(final.loc['2010', 'Medicaid'].mean(), 'plain')
aidlt = value_text(final.Medicaid.dropna().iloc[-1], 'plain')
aidvlt = tot.iloc[-1]
aidltdt = dtxt(tot.index[-1])['mon1']
txt2 = ('Medicare enrollment has grown from '+
        f'{are10} of the population in 2010 to {arelt}. Total Medicare '+
        f'enrollment is {arevlt:.1f} million in {areltdt}. Medicaid, which '+
        "was expanded in 2014, and the Children's Health Insurance Program "+
        f'(CHIP), increased enrollment from {aid10} of the population in 2010 '+
        f'to {aidlt} in {aidltdt}. The total enrollment for Medicaid and CHIP '+
        f'is {aidvlt:.1f} million people. ')
text = txt1 + '\n\n' + txt2
write_txt(text_dir / 'prog_sh.txt', text)
print(text)

The number of Social Security beneficiaries has increased from 53.4 million in 2010, or 17.2 percent of the population, to 66.9 million in October 2023, equivalent to 19.9 percent of the population. 

Medicare enrollment has grown from 15.3 percent of the population in 2010 to 19.8 percent. Total Medicare enrollment is 66.3 million in August 2023. Medicaid, which was expanded in 2014, and the Children's Health Insurance Program (CHIP), increased enrollment from 17.4 percent of the population in 2010 to 26.8 percent in August 2023. The total enrollment for Medicaid and CHIP is 90.0 million people. 


### Treasury Monthly Data

In [7]:
cols = ['record_date', 'classification_desc', 
        'current_month_rcpt_outly_amt']
fields = ','.join(cols)
lines = ','.join(['2', '3', '5', '6', '7', '12'])
url = ('https://api.fiscaldata.treasury.gov/services/api/'+
       'fiscal_service/v1/accounting/mts/mts_table_9'+
       f'?filter=src_line_nbr:in:({lines})&format=csv&'+
       f'fields={fields}&page[size]=700')

other = lambda x: x['Total'] - x['Individual Income Taxes']
si_cols = ['Employment and General Retirement', 'Other Retirement', 
           'Unemployment Insurance']
si = lambda x: x[si_cols].sum(axis=1)
df = (pd.read_csv(url, index_col=cols[:2], parse_dates=True)
        ['current_month_rcpt_outly_amt']
        .unstack().astype('float').divide(1_000_000_000_000)
        .assign(Other = other, SI = si).rolling(12).sum().dropna())
df.to_csv(data_dir / 'tmb_rec.csv', float_format='%g', 
          index_label='date')

In [8]:
df = pd.read_csv(data_dir / 'tmb_rec.csv', index_col='date', 
                 parse_dates=True)
# End nodes
d = {'Individual Income Taxes': 'blue!70!black', 
     'Other': 'blue!40!cyan'}
adj = node_adj(df[d.keys()])
dt = {n: None for n in d.keys()}
dt[df[d.keys()].iloc[-1].idxmax()] = 'm'
adj[df[d.keys()].iloc[-1].idxmax()] += 0.36
nodes = '\n'.join([end_node(df[n], c, date=dt[n], offset=adj[n]) 
                   for n, c in d.items()])
write_txt(text_dir / 'tmb_end_nodes.txt', nodes)
cl_iit = c_line(d['Individual Income Taxes'])
cl_oth = c_line(d['Other'])

# Text 
#url = 'https://fiscal.treasury.gov/reports-statements/mts/current.html'
dft = df.applymap(lambda x: f'\${x:.1f} trillion')
tval = dft.iloc[-1]
prval = dft['Total'].iloc[-13]
ltdt = dtxt(df.index[-1])['mon1']
prdt = dtxt(df.index[-13])['mon1']
ltsh = (df['Individual Income Taxes'].iloc[-1] / 
        df['Total'].iloc[-1]) * 100
tsh = value_text(ltsh, 'plain', digits=0)
text = (f'Over the 12 months ending {ltdt}, '+
        f'\\textbf{{federal government receipts}} total {tval["Total"]}, '+
        f'compared to {prval} one year prior, in {prdt}.\n\n'
        f'Over the past 12 months, {tsh} of receipts, totalling '+
        f'{tval["Individual Income Taxes"]}, are from '+
        f'individual income taxes {cl_iit}. The remaining receipts '+
        f'{cl_oth} are largely social insurance contributions '+
        f'({tval["SI"]}) and corporate income taxes '+
        f'({tval["Corporation Income Taxes"]}).')
write_txt(text_dir / 'tmb_rec.txt', text)
print(text)

Over the 12 months ending October 2023, \textbf{federal government receipts} total \$4.5 trillion, compared to \$4.9 trillion one year prior, in October 2022.

Over the past 12 months, 49 percent of receipts, totalling \$2.2 trillion, are from individual income taxes (see {\color{blue!70!black}\textbf{---}}). The remaining receipts (see {\color{blue!40!cyan}\textbf{---}}) are largely social insurance contributions (\$1.6 trillion) and corporate income taxes (\$0.5 trillion).


### Size of Treasury Market

In [9]:
url = ('https://api.fiscaldata.treasury.gov/services/api/fiscal_service/'+
       'v1/debt/mspd/mspd_table_1?sort=-record_date&fields=record_date,'+
       'security_type_desc,security_class_desc,total_mil_amt')

r = requests.get(url).json()

df = pd.DataFrame(r['data']).set_index('record_date')
ltdt = df.index[0]
ltval = df.query('security_type_desc == "Total Marketable"')['total_mil_amt'].iloc[0]
tdt = dtxt(ltdt)['mon1']
tval = f'\${float(ltval) / 1_000_000:.1f} trillion'

text = (f'As of {tdt}, the public holds {tval} in marketable treasuries. ')
write_txt(text_dir / 'treas_market.txt', text)
print(text)

As of November 2023, the public holds \$26.3 trillion in marketable treasuries. 


### Government Expenditures by Type

Longer-term data from OMB Historical Data Table 8.3

Shorter-term data from Treasury Monthly Statement Table 9

In [10]:
# Retrieve long-term data
url = 'https://www.whitehouse.gov/wp-content/uploads/2023/03/hist08z3_fy2024.xlsx'
data = pd.read_excel(url, header=[2,3,4], skipfooter=2, index_col=0)
data = data.rename(columns=lambda x: x if not 'Unnamed' in str(x) else '')
data.columns = [' '.join(col).strip() for col in data.columns.values]
data['Mandatory'] = data.loc[:,data.columns.duplicated()]
data = data.loc[:,~data.columns.duplicated()]
data = data.rename({'Total': 'Discretionary', '': 'Total'}, axis=1).loc['1989':]
data.columns = [i.replace('\n', ' ').replace('Mandatory Programmatic', 'MP') 
                for i in data.columns]
data.to_csv(data_dir / 'gov_exp_omb_raw.csv', index_label='date')

/home/brian/miniconda3/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [11]:
data = pd.read_csv(data_dir / 'gov_exp_omb_raw.csv', index_col='date')
res = data[['Net Interest', 'Non- defense', 'National Defense', 
            'MP Social Security', 'MP Medicare']].copy()
res['Social Safety Net'] = (data['MP Medicaid'] + 
                            data['MP Other Means  Tested Entitlements (1)'])
res['Other'] = 100 - res.sum(axis=1)
res = res.loc[~res.index.str.contains('estimate')]
res.index = pd.to_datetime(res.index) + pd.DateOffset(months=6)
res.to_csv(data_dir / 'gov_exp_omb.csv', index_label='date')

cols = ['National Defense', 'Non- defense', 'MP Medicare', 'MP Social Security', 
        'Social Safety Net', 'Other', 'Net Interest']
sdf = res[cols].iloc[-1]
height = ((sdf.cumsum() - (sdf / 2) + 4)).to_dict()
val = sdf.to_dict()
dt = dtxt(res.index[-1] + pd.DateOffset(weeks=2))['datetime']
nodes = [f'\\absnode{{{dt}}}{{{height[i]}}}{{\scriptsize {val[i]:.1f}\%}}' for i in cols]
dt2 = dtxt(res.index[-1] - pd.DateOffset(months=12))['datetime']
dv = dtxt(res.index[-1])['year']
dtnode = f'\\absnode{{{dt2}}}{{{108}}}{{\\footnotesize {dv}}}'
nodetext = '\n'.join([dtnode] + nodes)
write_txt(text_dir / 'gov_exp_nodes.txt', nodetext)

val19 = res.loc['2019-07-01']
val89 = res.loc['1989-07-01']
def19 = val19['National Defense']
def89 = val89['National Defense']
ni19 = val19['Net Interest']
ni89 = val89['Net Interest']
combcat = ['MP Medicare', 'MP Social Security', 'Social Safety Net']
comb19 = val19.loc[combcat].sum()
comb89 = val89.loc[combcat].sum()
colors = {'Defense': 'green!80!blue!68!black',
          'NonDef': 'green!78!black',
          'NI': 'orange!80!yellow',
          'SS': 'cyan!50!blue!95!white',
          'Medicare': 'blue!65!black',
          'SSN': 'cyan!95!white'}

cb = {k: c_box(v).replace('see ', 'see') for k,v in colors.items()}

text = (f'national defense spending fell to {def19:.1f} percent '+
        f'of outlays in 2019 from {def89:.1f} percent in 1989 '+
        f'{cb["Defense"]}. Discretionary non-defense '+
        'spending maintained a relative stable share of spending '+
        f'over the period {cb["NonDef"]}. Net interest '+
        'expense, the cost of federal borrowing, fell along with '+
        f'long-term interest rates, to {ni19:.1f} percent of outlays '+
        f'in 2019 from {ni89:.1f} percent in 1989 '+
        f'{cb["NI"]}. \n\n Offsetting the reduction in '+
        'spending on interest and national defense, Medicare and '+
        'Social Security now make up a larger share of federal '+
        'spending, as a larger share of people are retirement age. '+
        'Likewise, spending on the social safety net (means-tested '+
        'benefits and Medicaid) increased as employment-to-population '+
        'ratios fell and Medicaid was expanded. Medicare '+
        f'{cb["Medicare"]}, Social Security '+
        f'{cb["SS"]}, and the social safety net '+
        f'{cb["SSN"]} combine to comprise {comb19:.1f} '+
        'percent of federal spending in 2019, compared to '+
        f'{comb89:.1f} percent in 1989. ')
write_txt(text_dir / 'gov_exp_comp1.txt', text)
print(text)

national defense spending fell to 15.2 percent of outlays in 2019 from 26.6 percent in 1989 (see\cbox{green!80!blue!68!black}). Discretionary non-defense spending maintained a relative stable share of spending over the period (see\cbox{green!78!black}). Net interest expense, the cost of federal borrowing, fell along with long-term interest rates, to 8.4 percent of outlays in 2019 from 14.8 percent in 1989 (see\cbox{orange!80!yellow}). 

 Offsetting the reduction in spending on interest and national defense, Medicare and Social Security now make up a larger share of federal spending, as a larger share of people are retirement age. Likewise, spending on the social safety net (means-tested benefits and Medicaid) increased as employment-to-population ratios fell and Medicaid was expanded. Medicare (see\cbox{blue!65!black}), Social Security (see\cbox{cyan!50!blue!95!white}), and the social safety net (see\cbox{cyan!95!white}) combine to comprise 54.8 percent of federal spending in 2019, com

In [12]:
#Retrieve short-term data
cols = ['record_date', 'classification_desc', 
        'current_month_rcpt_outly_amt']
fields = ','.join(cols)
lines = ','.join([str(i) for i in list(range(14, 34))])
url = ('https://api.fiscaldata.treasury.gov/services/api/'+
       'fiscal_service/v1/accounting/mts/mts_table_9'+
       f'?filter=src_line_nbr:in:({lines})&format=csv&'+
       f'fields={fields}&page[size]=6000')
df = (pd.read_csv(url, index_col=cols[:2], parse_dates=True)
        ['current_month_rcpt_outly_amt']
        .unstack().astype('float').divide(1_000_000))
df.to_csv(data_dir / 'gov_exp_mts_raw.csv', index_label='date')

In [13]:
df = pd.read_csv(data_dir / 'gov_exp_mts_raw.csv', index_col='date', 
                 parse_dates=True)
oth = ['Administration of Justice', 'Agriculture',
       'Commerce and Housing Credit', 
       'Community and Regional Development',
       'Education, Training, Employment, and Social Services', 
       'Energy', 'General Government', 
       'General Science, Space, and Technology',
       'International Affairs', 'Natural Resources and Environment', 
       'Transportation', 'Undistributed Offsetting Receipts', 
       'Veterans Benefits and Services']
df['Other'] = df[oth].sum(axis=1)
res = df[['Income Security', 'Social Security', 'Health', 
          'National Defense', 'Medicare', 'Net Interest', 
          'Other']].copy().rolling(12).sum()
final = ((res.divide(res.sum(axis=1), axis=0) * 100).dropna())
final.loc['2016-12-01':].to_csv(data_dir / 'gov_exp_mts.csv', 
                                index_label='date')

cols = ['National Defense', 'Health', 'Medicare', 'Social Security', 
        'Income Security', 'Other', 'Net Interest']
sdf = final[cols].iloc[-1]
height = ((sdf.cumsum() - (sdf / 2) + 4)).to_dict()
val = sdf.to_dict()
dt = dtxt(final.index[-1] + pd.DateOffset(weeks=1))['datetime']
nodes = [f'\\absnode{{{dt}}}{{{height[i]}}}{{\scriptsize {val[i]:.1f}\%}}' 
         for i in cols]
dt2 = dtxt(final.index[-1] - pd.DateOffset(months=3))['datetime']
dv = dtxt(final.index[-1])['mon6']
dtnode = f'\\absnode{{{dt2}}}{{{108}}}{{\\footnotesize {dv}}}'
nodetext = '\n'.join([dtnode] + nodes)
write_txt(text_dir / 'gov_exp_nodes2.txt', nodetext)

ltdt = dtxt(final.index[-1])['mon1']
lt = final.iloc[-1]
incseclt = lt['Income Security']
maxdt = dtxt(final[['Income Security', 'Other']].sum(axis=1).idxmax())
maxmon = maxdt['mon1']
mx = final.loc[maxdt['datetime']]
pc = final.loc['2019-12-31']
incsecmx = mx['Income Security']
incsecpc = pc['Income Security']
text = ('Income security, which includes economic impact payments, the '+
        'child tax credit, unemployment compensation, food and nutrition '+
        'assistance, federal employee retirement and disability, and '+
        f'housing assistance, was {incseclt:.1f} percent of federal spending '+
        f'over the 12 months ending {ltdt} {c_box("brown")}. At its peak, '+
        f'over the 12 months ending {maxmon}, income security comprised '+
        f'{incsecmx:.1f} percent of federal spending. Pre-pandemic, in 2019, '+
        f'the category comprised {incsecpc:.1f} percent. ')
write_txt(text_dir / 'gov_exp_comp2.txt', text)
print(text)

cols2 = {'Income Security': 'brown', 'Health': 'violet!50!black', 
         'Medicare': 'blue!65!black', 
         'Social Security': 'cyan!50!blue!95!white', 
         'National Defense': 'green!80!blue!68!black',  
         'Net Interest': 'orange!80!yellow', 
         'Other: ': 'yellow!50!brown!55!white'}
col_names = {k: f'\hspace{{-1mm}}\cbox{{{v}}}{k}' for k, v in cols2.items()}
col_names2 = {i: f'\hspace{{6mm}}{i}' for i in oth}

data = df.rolling(12).sum()
data = data.divide(data.Total, axis=0) * 100
data = data.rename({'Other': 'Other: '}, axis=1)
data = data[list(cols2.keys()) + oth]
tbl = data.iloc[-3:].iloc[::-1].T
dmax = data.loc[maxdt['datetime']]
d19 = data.loc['2019'].mean().rename('2019')
d17 = data.loc['2017'].mean().rename('2017')
tbl = tbl.join(dmax)
tbl.columns = [dtxt(i)['mon2'] for i in tbl.columns]
tbl = tbl.join(d19).join(d17)
tbl = tbl.rename(col_names).rename(col_names2)
tbl.index = tbl.index.str.replace('and', '\&')
tbl = tbl.round(1).applymap('{:.1f}'.format)
tbl = tbl.rename({'\hspace{6mm}Education, Training, Employment, \& Social Services': 
            '\hspace{6mm}Educ., Training, Employment, \& Social Serv.'})
(tbl.to_csv(data_dir / 'gov_exp_comp.tex', sep='&', 
            lineterminator='\\\ ', quotechar=' '))

ltval = data['Other: '].iloc[-1]
maxval = data.loc[maxdt['datetime'], 'Other: ']
val19 = data.loc['2019', 'Other: '].mean()
text = (f"The category labeled ``other'' in the above-right chart "+
        f'includes several subcategories worth examining. The '+
        f'category increased to {ltval:.1f} percent of federal '+
        f'spending during the 12 months ending {ltdt}, from '+
        f'{maxval:.1f} percent during the 12 months ending {maxmon} '+
        f'{c_box(cols2["Other: "])}. Prior to the pandemic, in 2019, '+
        f'the category was {val19:.1f} percent of spending. ')
write_txt(text_dir / 'gov_exp_comp3.txt', text)
print('\n', text)

Income security, which includes economic impact payments, the child tax credit, unemployment compensation, food and nutrition assistance, federal employee retirement and disability, and housing assistance, was 12.3 percent of federal spending over the 12 months ending October 2023 (see\cbox{brown}). At its peak, over the 12 months ending March 2021, income security comprised 26.0 percent of federal spending. Pre-pandemic, in 2019, the category comprised 11.5 percent. 

 The category labeled ``other'' in the above-right chart includes several subcategories worth examining. The category increased to 12.9 percent of federal spending during the 12 months ending October 2023, from 24.2 percent during the 12 months ending March 2021 (see\cbox{yellow!50!brown!55!white}). Prior to the pandemic, in 2019, the category was 12.8 percent of spending. 


### Decomposition of US Public Debt Dynamics

In [14]:
url = ('https://www.federalreserve.gov/datadownload/Output.aspx?'+
       'rel=Z1&series=005f835d2ffea228372c2cc838173fa5&lastobs=&'+
       'from=01/01/1988&to=12/31/2023&filetype=csv&label=include&'+
       'layout=seriescolumn')
d, df = clean_fed_data(url)
n = {'FA316006005.A': 'Fiscal balance',
     'FA316130001.A': 'Interest paid',
     'FL314104005.A': 'DSL',
     'FL314190005.A': 'Liab',
     'FA315000905.A': 'Borrowing'}
data = df[n.keys()].rename(n, axis=1)

In [15]:
data['Primary balance'] = data['Fiscal balance'] + data['Interest paid']
gdpa = (nipa_df(retrieve_table('T10105A')['Data'], ['A191RC'])
        ['A191RC'])
data['GDP'] = gdpa
data['pi_'] = (nipa_df(retrieve_table('T10104A')['Data'], ['A191RG'])
               ['A191RG'])
data['defl'] = data['pi_'].iloc[-1] / data['pi_']
data['Real GDP'] = data['defl'] * data['GDP']

pb = -data['Primary balance']
i = data['Interest paid'] / data['DSL'].shift()
pi = data['pi_'].pct_change()
g = data['Real GDP'].pct_change()
lmbda = data['GDP'].pct_change()
d = data['DSL'] / data['GDP']
p = pb / data['GDP']

res = (((i / (1 + lmbda)) * d.shift()) - 
       ((pi / (1 + lmbda)) * d.shift()) - 
       ((g / (1 + g)) * d.shift()) +
       p)

data['calc'] = res

g2 = 0
res3 = (((i / (1 + lmbda - g)) * d.shift()) - 
       ((pi / (1 + lmbda - g)) * d.shift()) - 
       ((g2 / (1 + g2)) * d.shift()) +
       p)

res4 = (((pi / (1 + lmbda)) * d.shift()) - 
       ((pi / (1 + lmbda)) * d.shift()) - 
       ((g / (1 + g)) * d.shift()) +
       p)

res5 = (((i / (1 + lmbda)) * d.shift()) - 
       ((pi / (1 + lmbda)) * d.shift()) - 
       ((g / (1 + g)) * d.shift()))

data['e_gdp'] = (res - res3)
data['e_p'] = (res - res5)
data['e_r'] = (res - res4)
data['calc2'] = data['e_gdp'] + data['e_p'] + data['e_r']
data['sfa'] = d.diff() - data['calc2']
data['total'] = d.diff()
final = data[['e_gdp', 'e_p', 'e_r', 'sfa', 'total']] * 100
final = final.dropna()
final.index = final.index + pd.DateOffset(months=6)
final.to_csv(data_dir / 'debt_dynamics.csv', index_label='date')

col = {'p': 'blue!90!cyan!82!white',
       'gdp': 'violet!85!black', 'r': 'green!80!yellow!88!black', 
       'sfa': 'cyan!16!white'}

lt = final.iloc[-1]
ltdt = final.index[-1].year
totch = value_text(lt['total'], 'increase_by', 'pp', threshold=0.1)
pbch = value_text(lt['e_p'], 'contribution_to', 'pp', casual=True, threshold=0.1)
rch = value_text(lt['e_r'], 'contribution', 'pp', casual=True, threshold=0.1)
gdpch = value_text(lt['e_gdp'], 'contribution', 'pp', casual=True, threshold=0.1)
sfach = value_text(lt['sfa'], 'contribution', 'pp', casual=True, threshold=0.1)
cmb = lt[['e_gdp', 'e_p', 'e_r']].sum()
cmbtxt = 'greater' if cmb > d.diff().iloc[-1] else 'less'
text = (f'In {ltdt}, the debt to GDP ratio {totch} (see '+
        '\\begin{tikzpicture} \\node[line width=0.3mm, circle, '+
        'draw=black, scale=0.4, aspect=0.8] (d) at (0,0) {}; '+
        '\end{tikzpicture}). The primary balance '+
        f'{pbch} the debt to GDP ratio {c_box(col["p"])}, economic '+
        f'growth {gdpch} {c_box(col["gdp"])}, and real interest rates '+
        f'{rch} {c_box(col["r"])}. These combined factors were '+
        f'{cmbtxt} than the actual change in liabilities; '+
        'the adjustment to reconcile stocks and flows '+
        f'{sfach} {c_box(col["sfa"])}.')
write_txt(text_dir / 'debt_dynamics.txt', text)
print(text)

In 2022, the debt to GDP ratio decreased by 2.9 percentage points (see \begin{tikzpicture} \node[line width=0.3mm, circle, draw=black, scale=0.4, aspect=0.8] (d) at (0,0) {}; \end{tikzpicture}). The primary balance added 1.3 percentage points to the debt to GDP ratio (see\cbox{blue!90!cyan!82!white}), economic growth subtracted 1.9 percentage points (see\cbox{violet!85!black}), and real interest rates subtracted 4.1 percentage points (see\cbox{green!80!yellow!88!black}). These combined factors were less than the actual change in liabilities; the adjustment to reconcile stocks and flows added 1.8 percentage points (see\cbox{cyan!16!white}).


### Federal Interest Outlays share of GDP

In [16]:
gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])
gdp.index = gdp.index + pd.DateOffset(months=3)
gdp = gdp.A191RC.rolling(4).mean()
tot = fred_df('FYOINT')['VALUE']
tot.index = tot.index + pd.DateOffset(days=1)
df = (tot / gdp).dropna() * 100
fedadj = pd.read_csv(data_dir / 'fedintadj.csv', index_col='date', 
                     parse_dates=True)['RESPPLLOP_N.WW']
res = pd.DataFrame({'Tot': tot, 'FedAdj': fedadj}).fillna(0.0)
adj = res['Tot'] - res['FedAdj']
df2 = (adj / gdp).dropna() * 100
final = pd.DataFrame({'Tot': df, 'Adj': df2})

final.to_csv(data_dir / 'fedintexp.csv', index_label='date')
node = end_node(df, 'magenta', digits=2, date='fy', offset=0.15)
write_txt(text_dir / 'fedintexp_node.txt', node)
#node2 = end_node(df2, 'orange', digits=2)
#write_txt(text_dir / 'fedintexp_nodes.txt', '\n'.join([node, node2]))
val90s = df.loc['1990':'1999'].mean()

text = ('The Office of Management and Budget '+
        '\href{https://www.whitehouse.gov/omb/historical-tables/}{report} '+
        f'\\textbf{{federal interest outlays}} of \${tot.iloc[-1] / 1000:.0f} '+
        f'billion in fiscal year {tot.index[-1].year} (the year '+
        f'beginning October 1, {tot.index[-1].year -1}), compared to '+
        f'\${tot.iloc[-2] / 1000:.0f} billion in fiscal year '+
        f'{tot.index[-2].year}.\n\nPut into the context of the size of '+
        f'the economy, federal interest outlays in fiscal year '+
        f'{df.index[-1].year} were equivalent to {df.iloc[-1]:.2f} '+
        'percent of GDP (see {\color{magenta}\\textbf{---}}), following '+
        f'{df.iloc[-2]:.2f} percent of GDP in FY{df.index[-2].year} '+
        f'and {df.iloc[-3]:.2f} percent in FY{df.index[-3].year}, '+
        f'and compared to an average of {val90s:.1f} percent in '+
        'the 1990s, when interest rates were substantially higher. ')
write_txt(text_dir / 'fedintexp.txt', text)
print(text)

# Fed operating at loss (no remittances)
# val = (fedadj / 1_000).iloc[-1].round(-1)
# adjval = df2.iloc[-1]
# cl = c_line('orange')
# text = ('Often, the actual interest expense is slightly lower than the reported '+
#         'figure, because interest paid to the Federal Reserve gets returned '+
#         f'to the Treasury. In FY{df.index[-1].year}, '+
#         f'the Fed returned more than \${val:.0f} billion to the Treasury. '+
#         f'Adjusting for these remittances, the interest expense was '+
#         f'{adjval:.2f} percent of GDP in FY{df.index[-1].year} {cl}. ')
# write_txt(text_dir / 'fedintexp2.txt', text)
# print('\n', text)

The Office of Management and Budget \href{https://www.whitehouse.gov/omb/historical-tables/}{report} \textbf{federal interest outlays} of \$659 billion in fiscal year 2023 (the year beginning October 1, 2022), compared to \$475 billion in fiscal year 2022.

Put into the context of the size of the economy, federal interest outlays in fiscal year 2023 were equivalent to 2.44 percent of GDP (see {\color{magenta}\textbf{---}}), following 1.88 percent of GDP in FY2022 and 1.54 percent in FY2021, and compared to an average of 2.9 percent in the 1990s, when interest rates were substantially higher. 


### Public Debt by Holder

In [17]:
series = ['FDHBATN', 'GFDEBTN', 'FDHBFRBN', 'FDHBPIN', 'FDHBFIN']
start = '1988-01-01'
ftype = '&file_type=json'
base = 'https://api.stlouisfed.org/fred/series/observations?'

data = pd.DataFrame()

for srs in series:
    param = f'series_id={srs}&observation_start={start}&api_key={fred_key}'
    url = f'{base}{param}{ftype}'
    r = requests.get(url).json()['observations']
    s = pd.Series({pd.to_datetime(i['date']): (float(i['value']) / 1000.0) 
                   if srs in series[:2] else float(i['value']) for i in r})
    data[srs] = s
    
gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC']

In [18]:
# Add dot for latest value of total debt to GDP
totgdp = (data['GFDEBTN'] / (gdp / 1000)).dropna() * 100
ldtot = totgdp.index[-1]
ld = pd.to_datetime(data.dropna().index[-1])
ldt = dtxt(ldtot + pd.DateOffset(days=45))['datetime']
ltval = totgdp.iloc[-1]
lvt = f'{ltval:.1f}'
dt = dtxt(totgdp.index[-1])['qtr4']

# Only add mark to plot if necessary 
mark = ' (see \\tikz \draw[black, fill=black!20!white] (2.5pt,4pt) circle (2pt);)'
if ldtot > ld:
    node = (f'\\node[label={{[align=left]90:{{\scriptsize{dt}: '+
            f'\\\\ \scriptsize \ {lvt}}}}}, '+
            'circle, draw=black, fill=black!20!white, inner sep=1.4pt] at '+
            f'(axis cs:{ldt}, {ltval}) {{}};')
    write_txt(text_dir / 'pdebt_node.txt', node)
else:
    node = ''
    mark = ''
    write_txt(text_dir / 'pdebt_node.txt', node)

# Add text
lvls = data['GFDEBTN'].dropna().divide(1000).apply('\${:.1f} trillion'.format)
ltdt = dtxt(lvls.index[-1])['qtr1']
ltlvl = lvls.iloc[-1]
url = 'https://www.fiscal.treasury.gov/reports-statements/treasury-bulletin/current.html'
text = (f'Federal government public debt \href{{{url}}}{{totals}} '+
        f'{ltlvl} in {ltdt}, equivalent to {lvt} '+
        f'percent of GDP{mark}.')    
write_txt(text_dir / 'pdebt_ltval.txt', text)
print(text)

Federal government public debt \href{https://www.fiscal.treasury.gov/reports-statements/treasury-bulletin/current.html}{totals} \$33.2 trillion in 2023 Q3, equivalent to 120.0 percent of GDP.


In [19]:
df = data.copy().dropna()
df['PD'] = df['FDHBPIN'] - df['FDHBFIN']
df['IG'] = df['GFDEBTN'] - (df['FDHBFRBN'] + df['FDHBPIN'])
dgdp = df.loc['1989':].div(gdp / 1000.0, axis=0).dropna()
(dgdp * 100).to_csv(data_dir / 'pubdebt.csv', index_label='date')

# Text 
ldate = dtxt(ld)['qtr2']
ltgdp = dgdp['GFDEBTN'].iloc[-1] * 100
txt = 'Breaking down federal debt by holder, '
if ldtot > ld:
    txt = (f'In {ldate}, federal government public debt totals '+
           f'{lvls[df.index[-1]]}, equivalent to {ltgdp:.1f} '+
           'percent of GDP. Of this, ')
sh = df.div(df['GFDEBTN'], axis=0).iloc[-1] * 100
lv = df.iloc[-1] / 1000
text = (f'{txt}\${lv.PD:.1f} trillion, or {sh.PD:.1f} percent of '+
        'the total, is held by private domestic investors (see\cbox{green!60!black}). '+
        f'An additional \${lv.FDHBFIN:.1f} trillion, or {sh.FDHBFIN:.1f} percent '+
        'of the total, is held by foreign investors (see\cbox{orange!70!white}). '+
        'The remainder is held by the Federal Reserve (see\cbox{blue}) '+
        'and various government agencies and trusts (see\cbox{cyan!50!white}), '+
        'such as the Social Security Trust Fund. ')
write_txt(text_dir / 'pubdebt.txt', text)
print(text)

Breaking down federal debt by holder, \$13.8 trillion, or 41.5 percent of the total, is held by private domestic investors (see\cbox{green!60!black}). An additional \$7.6 trillion, or 22.9 percent of the total, is held by foreign investors (see\cbox{orange!70!white}). The remainder is held by the Federal Reserve (see\cbox{blue}) and various government agencies and trusts (see\cbox{cyan!50!white}), such as the Social Security Trust Fund. 
